In [1]:
%matplotlib qt

In [2]:
import yt
import matplotlib.pyplot  as plt
import numpy as np

import matplotlib as mpl

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
plt.rc('legend', fontsize=10)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_97629/2521444757.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [103]:
filedir = [
            # '/Users/yao/Desktop/Data/run1/job.83613/', # 4 beams
            # '/Users/yao/Desktop/job.85198/', # 2 beams
            # '/Users/yao/Desktop/job.85215/', # 1 beams
            '/Users/yao/Desktop/job.85216/', # 2 beams
          ] 

filename = 'lasslab_hdf5_chk_0030'  

In [104]:
def get_yt_data_1d(filename):
    j_idx = 0
    k_idx = 0
    fields_list = ["x", "dens", "tele", "tion",
                   "pres", "temp", "cham", "targ", 
                   "velx", "ye", "magz"]

    data_yt = yt.load(filename)
    data_yt_map = data_yt.covering_grid(
        level=0, left_edge=[data_yt.domain_left_edge[0], 
                            data_yt.domain_left_edge[1], 
                            data_yt.domain_left_edge[2]
                            ], 
                            dims=data_yt.domain_dimensions
    )
    time_now = np.array(data_yt.current_time)*1e9
    xx = np.linspace(data_yt.domain_left_edge[0]*1e4, 
                     data_yt.domain_right_edge[0]*1e4,
                     data_yt.domain_dimensions[0])

    data_dict = {}
    for f in fields_list:
        data_dict[f] = data_yt_map[f][:,j_idx,k_idx]
    return data_dict, time_now, xx

In [105]:
data_dict, time_now, xx = get_yt_data_1d(filedir[0]+filename)

yt : [INFO     ] 2025-01-29 18:09:25,056 Particle file found: lasslab_hdf5_chk_0030


yt : [INFO     ] 2025-01-29 18:09:25,083 Parameters: current_time              = 1.5000050153214534e-07
yt : [INFO     ] 2025-01-29 18:09:25,087 Parameters: domain_dimensions         = [128   1   1]
yt : [INFO     ] 2025-01-29 18:09:25,089 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-01-29 18:09:25,090 Parameters: domain_right_edge         = [6. 1. 1.]
yt : [INFO     ] 2025-01-29 18:09:25,091 Parameters: cosmological_simulation   = 0


In [106]:
width  = 3.487 * 1.5
height = width / 1.618 #* 0.8

fig, axs = plt.subplots(1, 1, sharex=True)

axs.semilogy(data_dict['x'], data_dict['dens'], '-k', lw=1.0)
axs.set_ylabel(r'Density (g/cm$^3$)', color='k', fontsize=16)
axs.tick_params(axis='y',color='k', labelcolor='k')
axs.set_ylim([1e-6,1e0])
axs.set_xlim([0,6])
axs.set_xlabel('x (cm)', fontsize=16)
axs.grid(which='major', alpha=0.5,linestyle=':', lw=1.0)

ax2 = axs.twinx()
ax2.plot(data_dict['x'], data_dict['tele']/11600, '-.r', label='Te', lw=1.5)
ax2.plot(data_dict['x'], data_dict['tion']/11600, '--b', label='Ti', lw=1.5)
ax2.set_ylabel('Temperatures (eV)', color='b', fontsize=16)
ax2.tick_params(axis='y', color='b', labelcolor='b')
ax2.spines['right'].set_color('b')
ax2.spines['left'].set_color('k')
ax2.set_ylim([0, 300])


fig.legend(['dens','Te','Ti'], 
           bbox_to_anchor=(0.6, 0.84),
           fontsize=12, 
           fancybox=False, 
           frameon=False)
# axs.set_title('single-beam case at {:.0f} ns'.format(time_now), fontsize=16)
axs.set_title('double-beam case at {:.0f} ns'.format(time_now), fontsize=16)
# axs.set_title('4 pulses')
fig.set_size_inches(width, height)
fig.tight_layout()



In [107]:
fig.savefig(filedir[0]+'profile_{:.0f}_2beam.pdf'.format(time_now),dpi=600)